# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq coinor-cbc



# **Ex1: Part 1 to 6** - *Problem Fomulation, Model, and Solver*


 \begin{array}{l}
Let\ x_{ij} \ be\ the\ number\ of\ truck\ load\ from\ the\ i^{th} \ warehouse\ to\ the\ j^{th} \ market\ \\
where\ i\ =1,2,...,8\ ;\ j=1,2...,8\\
\\
We\ are\ given\ the\ cost\ matrix\ as\ C\ where\ C_{ij} \ represnt\ the\ cost\ of\ transportin\ \\
from\ the\ i^{th} \ warehoue\ to\ the\ j^{th} \ market.\\
\\
Then\ our\ optimization\ problem\ becomes\\
\\
Min\ :\ \sum ^{8}_{i=1}\sum ^{8}_{j=1} C_{ij} \ x_{ij} \ \ \ \ ,\ i\ =1,2,...,8\ ;\ j=1,2...,8\\
\\
st.\\
\sum ^{8}_{j=1} x_{1j} \ \leqslant 100\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i1} \ =\ 75\\
\sum ^{8}_{j=1} x_{2j} \ \leqslant 250\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i2} \ =\ 300\ \\
\sum ^{8}_{j=1} x_{3j} \ \leqslant 200\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i3} \ =\ 250\\
\sum ^{8}_{j=1} x_{4j} \ \leqslant 200\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i4} \ =\ 200\\
\sum ^{8}_{j=1} x_{5j} \ \leqslant 150\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i5} \ =\ 400\\
\sum ^{8}_{j=1} x_{6j} \ \leqslant 90\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i6} \ =\ 100\\
\sum ^{8}_{j=1} x_{7j} \ \leqslant 290\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i7} \ =\ 50\\
\sum ^{8}_{j=1} x_{8j} \ \leqslant 200\ \ \ \ and\ \ \ \ \ \ \ \sum ^{8}_{i=1} x_{i8} \ =\ 70\\
\\
x_{ij} \ \in \ \mathbb{N} \cup \{0\} \ \ \ \ \ \ \ \ \ ,i\ =1,2,...,8\ ;\ j=1,2...,8\ \\
\end{array}






In [ ]:
#Creating a data frame from the csv
data = pd.read_csv('lab5_ex1.csv')
demand = np.array([75,300,250,200,400,100,50,70])

#Creating model, number of rows, 'M' and number of columns 'N' 
model = ConcreteModel()
N = len(data)
M = len(data.columns)-2

#Creating indexes for rows and columns
col_index = np.arange(N)
row_index = np.arange(M)

#Creating model variables, objective, and constrainsts
model.x = Var(row_index,col_index,domain=NonNegativeIntegers)
model.obj = Objective(expr = sum(model.x[i,j] * data.iloc[i,j+1] for i in row_index for j in col_index) , sense=minimize)
model.cons = ConstraintList()

#Demand Constraint
for k in col_index:
  model.cons.add( sum(model.x[i,k] for i in row_index) == demand[k] )

#Supply constraint    
for k in row_index:
  model.cons.add( sum(model.x[k,j] for j in col_index) <= data.iloc[k,9])

#Printing the model to verify the details
model.pprint()

#Solver and result
cbc_solver = SolverFactory('cbc')
result = cbc_solver.solve(model)


4 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :

# **Ex1: Part 7** - *Printing the solution, Cost and Variables*

The total cost and the number of truck load from ith warehouse to the jth market is as follows:

In [ ]:
print('The total transportation cost (Optimal Solution) : ', model.obj())
print('\n')

print('The number of truck loads transported are as follows: ',end='\n\n')
for i in row_index: 
  for j in col_index:
    if model.x[i,j].value > 0 :
      print('x[',i,',',j,'] : ', model.x[i,j].value,end="    i.e from the ")
      print(data.iloc[i,0],'to ', data.columns[j+1])

The total transportation cost (Optimal Solution) :  635720.0


The number of truck loads transported are as follows: 

x[ 0 , 1 ] :  100.0    i.e from the Ahmedabad to  Jodhpur
x[ 1 , 2 ] :  50.0    i.e from the Bengaluru to  Vellore
x[ 1 , 4 ] :  200.0    i.e from the Bengaluru to  Hyderabad
x[ 2 , 2 ] :  200.0    i.e from the Chennai to  Vellore
x[ 3 , 1 ] :  70.0    i.e from the Delhi to  Jodhpur
x[ 3 , 3 ] :  130.0    i.e from the Delhi to  Kanpur
x[ 4 , 5 ] :  80.0    i.e from the Kolkata to  Patna
x[ 4 , 7 ] :  70.0    i.e from the Kolkata to  Cuttack
x[ 5 , 3 ] :  70.0    i.e from the Lucknow to  Kanpur
x[ 5 , 5 ] :  20.0    i.e from the Lucknow to  Patna
x[ 6 , 0 ] :  75.0    i.e from the Mumbai to  Indore
x[ 6 , 1 ] :  130.0    i.e from the Mumbai to  Jodhpur
x[ 6 , 4 ] :  50.0    i.e from the Mumbai to  Hyderabad
x[ 7 , 4 ] :  150.0    i.e from the Nagpur to  Hyderabad
x[ 7 , 6 ] :  50.0    i.e from the Nagpur to  Raipur


# **Ex1: Part 8, 9** : *No transportation possible on Banglore-Patna Route*

One way to tackle this problen without changing the above model is to increase the cost of this route in the cost matrix to such an extent that the solver does not try to choose this model in any way.  

In [ ]:

#Changing the banglore patna cost to infinity
data.iloc[1,6] = 9999999999

#Creating a new model again

demand = np.array([75,300,250,200,400,100,50,70])

#Creating model, number of rows, 'M' and number of columns 'N' 
model2 = ConcreteModel()

N = len(data)
M = len(data.columns)-2

#Creating indexes for rows and columns
col_index = np.arange(N)
row_index = np.arange(M)

#Creating model variables, objective, and constrainsts
model2.x = Var(row_index,col_index,domain=NonNegativeIntegers)
model2.obj = Objective(expr = sum(model2.x[i,j] * data.iloc[i,j+1] for i in row_index for j in col_index) , sense=minimize)
model2.cons = ConstraintList()

#Demand Constraint
for k in col_index:
  model2.cons.add( sum(model2.x[i,k] for i in row_index) == demand[k] )

#Supply constraint    
for k in row_index:
  model2.cons.add( sum(model2.x[k,j] for j in col_index) <= data.iloc[k,9])


#Solver and result

result2 =cbc_solver.solve(model2)




***RESULT***

In [ ]:
print('The total transportation cost (Optimal Solution) : ', model2.obj())
print('\n')

print('The number of truck loads transported are as follows: ',end='\n\n')
for i in row_index: 
  for j in col_index:
    if model2.x[i,j].value > 0 :
      print('x[',i,',',j,'] : ', model2.x[i,j].value,end="    i.e from the ")
      print(data.iloc[i,0],'to ', data.columns[j+1])
  
      

The total transportation cost (Optimal Solution) :  635720.0


The number of truck loads transported are as follows: 

x[ 0 , 1 ] :  100.0    i.e from the Ahmedabad to  Jodhpur
x[ 1 , 2 ] :  50.0    i.e from the Bengaluru to  Vellore
x[ 1 , 4 ] :  200.0    i.e from the Bengaluru to  Hyderabad
x[ 2 , 2 ] :  200.0    i.e from the Chennai to  Vellore
x[ 3 , 1 ] :  70.0    i.e from the Delhi to  Jodhpur
x[ 3 , 3 ] :  130.0    i.e from the Delhi to  Kanpur
x[ 4 , 5 ] :  80.0    i.e from the Kolkata to  Patna
x[ 4 , 7 ] :  70.0    i.e from the Kolkata to  Cuttack
x[ 5 , 3 ] :  70.0    i.e from the Lucknow to  Kanpur
x[ 5 , 5 ] :  20.0    i.e from the Lucknow to  Patna
x[ 6 , 0 ] :  75.0    i.e from the Mumbai to  Indore
x[ 6 , 1 ] :  130.0    i.e from the Mumbai to  Jodhpur
x[ 6 , 4 ] :  50.0    i.e from the Mumbai to  Hyderabad
x[ 7 , 4 ] :  150.0    i.e from the Nagpur to  Hyderabad
x[ 7 , 6 ] :  50.0    i.e from the Nagpur to  Raipur
